# Lab 2A: Programming Paradigms in Python - Functional Programming


__Student:__ abcde123

# Functional Programming

Disclaimer: Functional programming in a language such as Python does not always lead to the fastest possible code and is often not considered the *pythonic* approach. However, functional programming is the basis for many concurrent systems (the MapReduce programming model many big data systems, e.g. Hadoop, rely on gets its name from the *map* and *reduce* functions mentioned below). Python is a multi-paradigmatic language, and functional programming is one of the paradigms one can use in the mix, to understand how and when to do this, it is necessary to do things in a non-*pythonic* way in order to cover the basics.

## General instructions

In this lab there are some general rules you should keep in mind to make sure you are on the correct path in your solutions.

#### Rules
1. You are not allowed to use `while` or `for` statements.
2. You are not allowed to use global variables (other than for functions defined in the global environment).
3. Code stubs should be viewed as fixed, you are only allowed to add code, the only code you are allowed to change is `pass` statements, which you should remove.
4. You should refrain from using the `list` datatype unless otherwise specified and instead use `tuple`. One of the strengths of functional programming is its focus on immutable data types (this is why functional programming and concurrency goes so well together).

#### Advice
1. Avoid local variables unless you are certain they are necessary, in most cases you won't need to use local variables. (altermatively, use local variables to your hearts content, but when your solution works, try to eliminate them, you should be able to eliminate most of them, over time, you might find that you don't need them.)

## 1 Recursion

Recursion is the main way of creating repeated operations in functional programming and consist of functions calling themselves, instead of using iterative constructs such as `while`- or `for`-loops which implicity require some kind of state. This is generally a more mathematical way of defining operations and many famous mathematical functions, such as the Fibonacci function, have a recursive definition, even though they usually can be implemented iteratively.

### 1.1 `triangle`

Below, you find a traditional iterative implementation of the _triangle_ function (https://en.wikipedia.org/wiki/Triangular_number) and a function stub for a recursive version. Write code to implement the recursive version of `triangle`.

[Note: Below, and in further code stubs, I use the `pass` statement. The pass statement is simply a non-action, it allows me to write a code stub without breaking syntax rules (a colon must be followed by a row with increased indentation).]<br>
[Literature: LP section 4, especially chapter 16 and 19.]

In [4]:
# Iterative version
def triangleiter(n):
    result = 0
    for x in range(n+1):
        result = result + x
    return result

# Recursive version
def trianglerec(n):
    if n == 0:
        return 0
    else:
        return n+trianglerec(n-1)

n = 5
print(triangleiter(n))
print(trianglerec(n))

15
15


### 1.2 `myflatten`

One common use of recursion is to _flatten_ recursive data structures, such as nested lists or tuples. This is relatively simple with only one level of nesting, or when the structure follows a strict pattern, but for arbitrary nested sequences, a recursive approach is more natural. Implement a recursive function `myflatten` which can take an arbitrary structure of nested tuples and flattens it.

In [2]:
from functools import reduce

def myflatten(l):
        return reduce(lambda a,b: a + (myflatten(b) if hasattr(b, '__iter__') else [b]), l, [])

   

myflatten((1, (2), 3, (4, 5, (6), 7), 8))

[1, 2, 3, 4, 5, 6, 7, 8]

### 1.3 Maximum recursion depth?

While Python is not a "pure" functional language like Haskell, or even a "fully" functional language like the Lisp family, it does implement a number of traditionally functional features. What it does lack however, is _Tail Call Optimization_ (_TCO_) which makes recursive algorithms much cheaper to run. Without going into too much detail, with _TCO_ the space complexity (how much of the computers memory we use) of simpler recursive algoritmhs is constant, which means we can theoretically recurse forever. However, without _TCO_, recursion has linear space complexity, which means that there is a limit to how many recursive calls we can make before we run out of memory even for simple algorithms. This means that one of the main features of functional programming, recursion, is not as useful in Python as in more functional languages.

Even though Python is not as efficient at doing recursion, a basic understanding of recursive algorithms is good to have and helps one get into the functional mindset.

Now that you have a recursive version of triangle, we are going to break it. First, run the code in the box below

In [ ]:
print(triangleiter(10000))
print(trianglerec(10000))

As you can see, we get a `RuntimeError` (or `RecursionError` in Python versions 3.5 and later), which tells us that "maximum recursion depth exceeded...". This means that we have reached that limit mentioned above, we have run out of memory (or rather, Python has capped the number of recursive calls we can make, in order to NOT run out of memory). But how deep can we go?

In this case, the largest possible input to `trianglerec` should be equivalent to the maximum recursion depth. There is an easiy way to compute the maximum recursion depth, using a single recursive function and exception handling. Create such a function below and run it to find the maximum recursion depth. Try it as input to `trianglerec`, does it work? Try the next larger number, does that work?

In [8]:

def test_recursion_depth(n, counter=0):
    try:
        counter += 1
        return test_recursion_depth(n, counter)
    except RuntimeError:
        return counter
    

    
n = test_recursion_depth(trianglerec)
print(n)
print(trianglerec(n-1))

2974
4420850


## 2 Higher-order functions and anonymous functions

A _higher-order function_ is a function which operates on other functions. What this means exactly is disputed, one definition is that a higher-order function must itself return a function, another definition is that a higher-order function must take another function as input. We will play fast and loose with the definition and call any function which does either or both of these a higher-order function. (Conversely, a function neither taking another function as input nor returning a function we will refer to as a _first-order function_) In R you have encountered these when, for instance, using the `apply` family of functions, which are all versions of what is called a `map` function in functional programming (see below).

When using higher-order functions, it is often useful to create simple anonymous functions at the place in the code where they are used, rather than defining a new named function in one place only to call it in a single other place. In R, all functions are created in this way with the `function` keyword, but they are usually assigned to global names with standard assignment (`<-`). Python provides similar functionality using the `lambda` keyword (name inspired by Alonzo Church's $\lambda$-calculus which has inspired much of functional programming) with which we can create anonymous functions. Of course, we can also pass named functions to higher-order functions, which is usually the case when the function is predefined, general enough to be used in more than one place, or complex enough to warrant separate definition and documentation for the sake of clarity.

### Standard higher-order functions: map, filter, reduce

There are three standard cases which are widely applicable and many other higher-order functions are special cases or combinations of these. They are: `map`, apply a function on each element in a sequence, `filter`, keep (or conversely, remove) elements from a sequence according to some condition, and `reduce`, combine the elements in a sequence. The `map` function takes a sequence and a function (usually of 1 parameter) which is to be applied to each element of the sequence and might return anything, this function is assumed not to have side effects. The `filter` function takes a function (usually of 1 parameter) which returns a boolean value used to indicate which elements are to be kept. The `reduce` function takes a function (usually of 2 parameters) which is used to combine the elements in the sequence.

In Python, `map` and `filter` are standard functions, available in the global environment. However, since Python 3, the `reduce` function has been moved to the built-in `functools` module and needs to be imported before we can use it.

Many more advanced functions, of any order, can be created by combining these three higher-order functions.

Usually, the `reduce` function is more difficult to grasp than `map` and `filter` but I found this blog-post by AndrÃ© Burgaud to be a nice introduction to `reduce`. Note that Burgaud talks about the more general _fold_ concept rather than `reduce`, which is a special case of fold often called _left fold_ (this is covered in more detail in the post). https://www.burgaud.com/foldl-foldr-python/

### 2.1 `mysum`

Implement a function `mysum` which computes the sum of a list or tuple of numbers using the reduce function and a lambda function.

In [6]:
from functools import reduce

def mysum(l):
    return reduce(lambda l, y: l+y,l)

mysum((4, 7, 1))

12

### 2.2 `mylength`

Implement a function `mylength` which uses `map` and `reduce` to compute the length of a sequence. The use of the `len` function is not allowed.

[Hint: Use `map` to convert the input to something which can easily be `reduce`:d.]

In [10]:
def mylength(l):
    return reduce(lambda x, y: x+y,list(map(lambda x: 1, l)))
    
print(mylength((4, 2, 5, 2, 5)))
print(mylength("test"))

5
4


### 2.3 mylength2

Implement a function `mylength` which uses only `reduce` to compute the length of a sequence using the optional third parameter to supply an initial value to `reduce`.

In [11]:
def mylength2(l):
    return reduce(lambda x, y: x+1 , l ,0)

print(mylength2((4, 2, 5, 2, 5)))
print(mylength2("test"))

5
4


### 2.4 mysieve

A very well known algorithm for finding prime numbers is the Sieve of Eratosthenes. This algorithm works by filtering  out all non-prime numbers up to some predefined $n$. Use the skeleton code below and write a recursive `inner` function using `filter` to find the prime numbers up to `n`.

[Note: In Python, the `filter` function returns a type of generator, if you want to convert this to a tuple, simply use `tuple(filter(...))`.]<br>
[Hint: Each call to `inner` should filter out the numbers divisible by the first element in `l`.]

In [15]:
def mysieve(n):
    def inner(l):
        if(len(l)>1):
            return [int(l[0])]+inner(tuple(filter(lambda x: x % l[0] != 0,l)))
        else:
            return [int(l[0])]
    return inner(tuple(range(2, n+1)))
a = mysieve(100)
a

[2,
 3,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79,
 83,
 89,
 97]

### Comprehensions

One popular feature in many functional programming languages is comprehensions, which we already covered in the first lab. However, comprehensions are definitely a part of the functional side of Python, oh, and they are considered *Pythonic* as well as being very fast.

Comprehensions can be used to simulate the behavior of `map` and `filter` (though not the behavior of `reduce`).

### 2.5 mysieve2

Reimplement the `mysieve` function using a comprehension instead of `filter`.

In [16]:
def mysieve2(n):
    def inner(l):
        if(len(l)>1):
            return (l[0],)+inner([x  for x in l if x%l[0] != 0])
        else:
            return (l[0],)
    return inner(tuple(range(2, n+1)))
mysieve2(100)

(2,
 3,
 5,
 7,
 11,
 13,
 17,
 19,
 23,
 29,
 31,
 37,
 41,
 43,
 47,
 53,
 59,
 61,
 67,
 71,
 73,
 79,
 83,
 89,
 97)

## 3 Building your own higher order functions

### 3.1 Reimplementing map, filter and reduce

Re-implement the three basic functional helper functions `map`, `filter` and `reduce` as recursive functions. Note that the built-in versions of these functions work on multiple sequences of equal length if supplied, however, you can assume a single sequence as second parameter, i.e. you can also skip the third parameter to reduce.

In [17]:
def mymap(fun, l):
    if(len(l)>1):
        return [fun(l[0])]+ mymap(fun, l[1:])
    else:
        return [fun(l[0])]
    

mymap(lambda x:x**2, tuple(range(10)))

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [19]:
def myfilter(fun, l):
    if(len(l) < 1):
        return []
    elif fun(l[0]):
        return [l[0]] + myfilter(fun, l[1:])
    else:
        return myfilter(fun,l[1:])
    
myfilter(lambda x:x%2==0, tuple(range(10)))

[0, 2, 4, 6, 8]

In [7]:
def myreduce(fun, l):
     if l==():
        return ()
     if len(l)==2:
        return fun(l[0],l[1])
     else:
        return fun(myreduce(fun,l[:(len(l)-1)]),l[-1])


print(reduce(lambda x, y: x*y, tuple(range(1,6))))
print(myreduce(lambda x, y: x*y, tuple(range(1,6))))

120
120


##  4 Returning functions

The previous section covered functions which take other functions as input, but what about the opposite, functions returning functions as output?

### 4.1 myidentity

This might seem like a trivial example but serves as a nice entry point into returning functions. Create a function `myidentity` wich takes a single input. When called `myidentity` should return a function which, when itself called, regardless of inputs, produces the original input to `myidentity`. This is useful for when we need to pass a function to a higher order function but want it only to produce a constant value, for instance the function passed to `map` when we implemented `mylength` above.

That the returned function should take arbitrary inputs means that we need to use the special \* and \*\* parameters, most often seen as `*args` and `**kwargs`. These are used to handle arbitrary numbers of unnamed or named arguments to functions. These are either used when a function might itself take an arbitrary number of inputs or when we want to pass along arguments from a higher order function to a concrete function.

In [134]:
def myidentity(x):
    def ret(*argv, **kwargs):
        return (x, argv, kwargs)

    return ret

one = myidentity(1)
print(one)
print(one(1,3,5, param2="a string", param3=[1, 2, 4]))

<function myidentity.<locals>.ret at 0x7f8479c8bd08>
(1, (1, 3, 5), {'param3': [1, 2, 4], 'param2': 'a string'})


### 4.2 mycomposite

Function composition is a common in both math and programming. Write a function `mycomposite` which takes two functions, $a$ and $b$, and produces the _composite_ function $a \circ b$, where $(a \circ b)(x) \Leftrightarrow a(b(x))$. Example use is given below.

In [28]:
from statistics import stdev, mean

def mycomposite(a, b):
    return lambda x: a(b(x))

def myscale(l):
    return [x/stdev(l) for x in l]

def myshift(l):
    return [x-mean(l) for x in l]

standardize = mycomposite(myscale, myshift)

print(standardize(range(-3, 8)))

[-1.507556722888818, -1.2060453783110545, -0.9045340337332909, -0.6030226891555273, -0.30151134457776363, 0.0, 0.30151134457776363, 0.6030226891555273, 0.9045340337332909, 1.2060453783110545, 1.507556722888818]


### 4.3 The pipeline function

When doing data analysis, one very important part is pre-processing. Often, data goes through a number of steps of preprocessing, sometimes called a pipeline. The function composition example above can be seen as a special case of such a pipeline for only two functions. By clever use of higher order functions, we can build a pipeline function which takes a list or tuple of data transforming functions and creates a function which applies these sequentially. Construct such a function called `make_pipeline`. In order to focus on the primary purpose of the `make_pipeline` function, we will perform a very simple set of transformations, increment each value by 1, take the absolute value, and then take the square root. Usage example and code for the `inc` function is supplied below.

In [126]:
import math

def make_pipeline(*funs):
    if not None in funs:
        return lambda x: reduce(lambda x,y: map(y,x), funs, x)

def inc(x):
    return x+1

def sqrt(x):
    return math.sqrt(x)

pipeline = make_pipeline(inc, abs, sqrt)


tuple(pipeline(range(-5,5)))


(2.0,
 1.7320508075688772,
 1.4142135623730951,
 1.0,
 0.0,
 1.0,
 1.4142135623730951,
 1.7320508075688772,
 2.0,
 2.23606797749979)

### 4.4 mypartial (Updated 2018-02-06)

One of the main arguments for functional programming is the lack of state. However, sometimes we need to be able to set some constants without passing around enormous sets of inputs to functions. One approach to this is the `partial` function in the `functools` module. The `partial` function takes a function and an arbitrary number of named arguments and returns a function with those arguments bound to the corresponding named parameters of the function, i.e. creating a function of fewer parameters. Implement your own version of the `partial` function.

[Note: In this case, you will have one dictionary of named  parameters(\*\*-parameters) for the `mypartial` function and one tuple of unnamed parameters (\*-parameters) and one dictionary of named  parameters(\*\*-parameters) for the the function you will return. Note that the \*\*-dictionaries cannot share the same name.]

In [9]:
def add(a, b):
    return a + b

def mypartial_old(func,**kwargs):
    def funct(*param,**nparam):
        return(func(*(param), **kwargs))
    return(funct)

# FIXED
def mypartial(func,**kwargs):
    def funct(*param,**nparam):
        namedParams = kwargs.copy()
        namedParams.update(nparam)
        return(func(*(param), **kwargs))
    return(funct)

add3 = mypartial(add, b = 3)
add3(10)

13

## 5 Closures, returning functions with state

When a function is created it always has access to the environment in which it was created. Usually, this means that the function can access variables in the global environment. However, whenever a function is created inside another function, it has access to the environment of the function in which it is created. To access such variables, they have to be declared, in the inner function, using the `nonlocal` keyword. If we return this inner function, we have created a _closure_.

A closure is a function which has access to an environment, not accessible from outside the function, but which is not destroyed when the function returns. I.e. it is a way to introduce a small measure of statefulness into functional programming. In Python, iterators and generators work much like this. However, we can use the general concept in many cases.

### 5.1 make_counter

Implement a function `make_counter` which has a single parameter `n` which acts as the initial value for a counter. The function should return a function with no parameters which, when called, increments the value of `n` by 1 and returns the new value.

In [140]:
def make_counter(n):
    
    def counter():
        nonlocal n
        n = n+1
        return n
    return counter

c = make_counter(0)
print(c())
print(c())
print(c())
print(c())
print(c())

1
2
3
4
5


### 5.2 Message passing: `make_counter2`

Often, it is beneficial to be able to inspect the state of a closure without changing that state or be able to change that state in more than one way. This can be achieved using _message passing_, i.e. letting the closure take one or more arguments which controls its behavior when called.

Implement a function `make_counter2` which has a single parameter `n` which acts as the initial value for a counter. The function should return a function which takes a single argument `message`. The `message` should be one of the strings "increment" or "decrement". On "increment" the value of `n` should increase by 1 and be returned, on "decrement" the value of `n`should decrease by 1 and be returned, for any other message (or no message) the value of `n` should be returned.

[Note: You need to use a default value for the `message` parameter so that the counter can be called without argument to return the current value of `n`.]

In [147]:
def make_counter2(n):
    def counter(s="0"):
        nonlocal n
        if s == "increment":
            n = n+1
        elif s =="decrement":
            n = n-1
        else:
            n
        return n
    return counter

c = make_counter2(0)
print(c())
print(c("increment"))
print(c("increment"))
print(c("increment"))
print(c("decrement"))
print(c())

0
1
2
3
2
2


## 6 Practical applications: Quicksort

### 6.1 Basic Quicksort (Updated 2018-05-06)

One of the most famous recursive functions is the Quicksort function (https://en.wikipedia.org/wiki/Quicksort). It allows us to sort a sequence, with repeated values, in log-linear time and with a logarithmic number of recursive calls. We will start by implementing Quicksort for a tuple of numbers.

You should note that Wikipedia illustrates a more advanced _in-place_ version of Quicksort. This means that the partition function is more advanced and that the quicksort function has 3 parameters instead of 1. For the purposes of this assignment you can simply pass a new tuple to each recursive call to quicksort (i.e. you can use _filter_ or a comprehension to create the inputs).

In [25]:
from random import sample, choice

    
def quicksort(qslist):
    """"Quicksort implementation."""
    if len(qslist) < 2:
        return qslist

    pivot = choice(qslist)
    small = [x for x in qslist if x < pivot]
    equal = [x for x in qslist if x == pivot]
    big = [x for x in qslist if x > pivot]

    return quicksort(small) + equal + quicksort(big)
 
    
    
a = tuple(sample(range(1000), 1000))
print(a)
b = quicksort(a)
print(b)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

### 6.2 Quicksort as a higher order function

The version of quicksort implemented above should only work on simpler types of data. However, what if we wanted to change the sort from ascending to descending, or we wanted to sort a tuple of objects where Python itself does not know how to compare object A and object B? To add this functionality, we can pass another argument to Quicksort which is applied to each element to compute a value on which it can be sorted. With the same function, only using different key-functions supplied, you should be able to sort a tuple descending, and sort a tuple of tuples of numbers on the sum of each tuple of numbers.

[Hint: To generate test data for the second case, you can use a tuple comprehension within a tuple comprehension together with the random module.]

In [89]:
from random import random

# Write quicksort2 here:
def quicksort2(qslist, fun, desc="normal"):
    def quicksort(qslist):
        """"Quicksort implementation."""
        if len(qslist) < 2:
            return qslist

        pivot = choice(qslist)
        small = [x for x in qslist if fun(x) < fun(pivot)]
        equal = [x for x in qslist if fun(x) == fun(pivot)]
        big = [x for x in qslist if fun(x) > fun(pivot)]

        return quicksort(small) + equal + quicksort(big)
    
    #qslist = tuple(map(lambda x: fun(x), qslist))

    return quicksort(qslist)

a = tuple(tuple(random() for i in range(3)) for j in range(10))
print(a)
b = quicksort2(a, sum)
print(b)

((0.46629596104845694, 0.9422863036182159, 0.3201662890152843), (0.10849091786532872, 0.27726159007347173, 0.38316876531683464), (0.6227267395296162, 0.0507004914811775, 0.4670459528655688), (0.21764225276012872, 0.31220574201202544, 0.5184975094817832), (0.48274770210535767, 0.8968722968423237, 0.015469854765951085), (0.10089114265468546, 0.14002714006855999, 0.23238962543629815), (0.008289818551792005, 0.4768879088930589, 0.2411866054219587), (0.7585033719940958, 0.6579479179909847, 0.9560046967141381), (0.7405069979238077, 0.5735689950341348, 0.7663680186464132), (0.5952327262143547, 0.3697988096414656, 0.024737401219847466))
[(0.10089114265468546, 0.14002714006855999, 0.23238962543629815), (0.008289818551792005, 0.4768879088930589, 0.2411866054219587), (0.10849091786532872, 0.27726159007347173, 0.38316876531683464), (0.5952327262143547, 0.3697988096414656, 0.024737401219847466), (0.21764225276012872, 0.31220574201202544, 0.5184975094817832), (0.6227267395296162, 0.0507004914811775,

### 6.3 Quicksort as a higher order function with unknown arguments

The version of Quicksort above only works with functions which takes no arguments but the object being compared. However, a higher order function can be made to pass along unnamed or named arguments using \*args or \*\*kwargs. Below you find a function implementing the probability density function for the Gaussian distribution, taking an observation x, as well as the location and scale for the distribution at hand, and computes the density at x.

We would now like to sort a tuple of observations on their corresponding density on the standard normal distribution. To do this, it is not enough to only pass `gausspdf` to quicksort but we also need to pass the values for `loc` and `scale`.

In [124]:
from math import exp, sqrt, pi
from random import uniform

def gausspdf(x, loc, scale):
    return (exp(-((x - loc) / scale)**2/2)/sqrt(2*pi))/scale

# Write quicksort3 here:
def quicksort3(qslist, fun, **kwargs):
    def quicksort(qslist):
        """"Quicksort implementation."""
        if len(qslist) < 2:
            return qslist

        pivot = choice(qslist)
        small = [x for x in qslist if fun(x,**kwargs) < fun(pivot,**kwargs)]
        equal = [x for x in qslist if fun(x,**kwargs) == fun(pivot,**kwargs)]
        big = [x for x in qslist if fun(x,**kwargs) > fun(pivot,**kwargs)]

        return quicksort(small) + equal + quicksort(big)
    
    #qslist = tuple(map(lambda x: fun(x), qslist))

    return quicksort(qslist)



    
    
a = tuple(uniform(-1,1) for i in range(10))
print(a)
b = quicksort3(a, gausspdf, loc=0, scale=1)
print(b)

(-0.5286049525094949, 0.37660073939715666, -0.7459257927343363, 0.31296999289836847, 0.2654334513796721, 0.2430320850902603, -0.0962914695506214, 0.9225021227262422, 0.11925923555113394, 0.8070889642558834)
[0.9225021227262422, 0.8070889642558834, -0.7459257927343363, -0.5286049525094949, 0.37660073939715666, 0.31296999289836847, 0.2654334513796721, 0.2430320850902603, 0.11925923555113394, -0.0962914695506214]


Of course, in many cases we could use `partial` to the same effect.

### 6.4 Quicksort for any type of sequence

All versions of quicksort above works with the tuple data type, but what if we wanted the return type to be a list, or a custom sequence type? Well, create a version of quicksort which adds a parameter which is used to supply a function which controls the type of the returned and sorted sequence.

[Bonus: Create a version which does not take an extra argument but simply uses the supplied sequence type.]

In [128]:
from math import exp, sqrt, pi
from random import uniform

def gausspdf(x, loc, scale):
    return (exp(-((x - loc) / scale)**2/2)/sqrt(2*pi))/scale

# Write quicksort4 here:
def quicksort4(qslist, typ, key, **kwargs):
    fun=key
    def quicksort(qslist):
        """"Quicksort implementation."""
        if len(qslist) < 2:
            return qslist

        pivot = choice(qslist)
        small = [x for x in qslist if fun(x,**kwargs) < fun(pivot,**kwargs)]
        equal = [x for x in qslist if fun(x,**kwargs) == fun(pivot,**kwargs)]
        big = [x for x in qslist if fun(x,**kwargs) > fun(pivot,**kwargs)]

        return quicksort(small) + equal + quicksort(big)
    
    #qslist = tuple(map(lambda x: fun(x), qslist))

    return typ(quicksort(qslist))



a = tuple(uniform(-1,1) for i in range(10))
print(a)
b = quicksort4(a, tuple, key=gausspdf, loc=0, scale=1)
print(b)
a = [uniform(-1,1) for i in range(10)]
print(a)
b = quicksort4(a, list, key=gausspdf, loc=0, scale=1)
print(b)

(0.5537961535970068, -0.5736787859296668, -0.5140632434569017, 0.6221556709523575, -0.5207058818630492, 0.35233817771508247, -0.2563018078590986, 0.3762498627379329, 0.6828550960383013, 0.8182484062896862)
(0.8182484062896862, 0.6828550960383013, 0.6221556709523575, -0.5736787859296668, 0.5537961535970068, -0.5207058818630492, -0.5140632434569017, 0.3762498627379329, 0.35233817771508247, -0.2563018078590986)
[0.557410794989512, 0.571237996870239, 0.5343314139659261, 0.17032714577912245, 0.19064653934664788, 0.7294115873246425, 0.644707127606772, 0.20273157513311557, -0.4053253605315448, 0.9735189745707244]
[0.9735189745707244, 0.7294115873246425, 0.644707127606772, 0.571237996870239, 0.557410794989512, 0.5343314139659261, -0.4053253605315448, 0.20273157513311557, 0.19064653934664788, 0.17032714577912245]
